<a href="https://colab.research.google.com/github/nikkizhou/Join-The-Queue/blob/main/MT_Nikki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install transformers

In [ ]:
%pip install -q bitsandbytes datasets accelerate loralib

Note: you may need to restart the kernel to use updated packages.


Fine-tuning LLMs with PEFT and LoRA
https://www.youtube.com/watch?v=Us5ZFp16PaU

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import torch

# Check if CUDA (GPU) is available
if torch.cuda.is_available():
    print("GPU is available!")
else:
    print("No GPU found.")

In [ ]:
# Freezing the original weights:

for param in model.parameters():
    param.requires_grad = False  # freeze the model - train adapters later
    if param.ndim == 1:
        # cast the small parameters (e.g. layernorm) to fp32 for stability
        param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
    def forward(self, x): return super().forward(x).to(torch.float32)

model.lm_head = CastOutputToFloat(model.lm_head)

In [ ]:
# Setting up the LoRa Adapters
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()

    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable: {100 * trainable_params / all_param}%"
    )


In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]= "0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM,BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_8bit=True)

model = AutoModelForCausalLM.from_pretrained(
    #  "bigscience/bloom-560m",
    "bigscience/bloom-7b1",
    #quantization_config=quantization_config,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")


model.safetensors.index.json:   0%|          | 0.00/28.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

ValueError: You are trying to offload the whole model to the disk. Please use the `disk_offload` function instead.

In [ ]:

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Sample data (you can replace this with your actual dataset)
data = {
    'text': ["This is a question", "How are you?", "What is your name?",
             "Tell me about yourself", "What is your favorite color?"],
    'label': [0, 1, 1, 0, 1]  # Sample labels
}
df = pd.DataFrame(data)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df['text'], df['label'], test_size=0.2, random_state=42)

# Text preprocessing with TF-IDF
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Define models
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),
    'SVM': SVC(kernel='linear'),
    'Random Forest': RandomForestClassifier(n_estimators=100)
}

# Train, predict, and evaluate models
for name, model in models.items():
    model.fit(X_train_tfidf, y_train)
    y_pred = model.predict(X_test_tfidf)

    # Evaluate model
    print(f"Model: {name}")
    print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print("Classification Report:")
    print(classification_report(y_test, y_pred, zero_division=0))
    print("-" * 50)

Model: Logistic Regression
Accuracy: 0.0000
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

--------------------------------------------------
Model: SVM
Accuracy: 0.0000
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       0.0
           1       0.00      0.00      0.00       1.0

    accuracy                           0.00       1.0
   macro avg       0.00      0.00      0.00       1.0
weighted avg       0.00      0.00      0.00       1.0

--------------------------------------------------
Model: Random Forest
Accuracy: 0.0000
Classification Report:
              precision    recall  f1-score   support

          

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier



from transformers import AutoTokenizer


mLPClassifier = MLPClassifier()
randomForestClassifier = RandomForestClassifier()




# LabelEncoder

In [ ]:
from sklearn.preprocessing import LabelEncoder

labels = ['positive', 'negative', 'neutral', 'positive', 'negative']
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

# Display original and encoded labels
print("Original Labels: ", labels)
print("Encoded Labels: ", encoded_labels)

# To decode the numerical labels back to original form
decoded_labels = label_encoder.inverse_transform(encoded_labels)
print("Decoded Labels: ", decoded_labels)

Original Labels:  ['positive', 'negative', 'neutral', 'positive', 'negative']
Encoded Labels:  [2 0 1 2 0]
Decoded Labels:  ['positive' 'negative' 'neutral' 'positive' 'negative']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# BERT for question type classification

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

#### 1. Load pre-trained BERT model and tokenizer

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Categorized_mocks.xlsx to Categorized_mocks.xlsx


In [ ]:
%pip install datasets

#### 2. Tokenize the Dataset

In [ ]:
from transformers import DataCollatorWithPadding,AutoTokenizer, AutoModelForSequenceClassification
from datasets import load_dataset
import pandas as pd

# 1. Get and reorgnize dataframe
excel_file = 'Categorized_mocks.xlsx'
df = pd.read_excel(excel_file, header=1)
df = df.iloc[:, :18]
df.iloc[3:,3:] = df.iloc[3:,3:].fillna(0)

# 2. load dataset
csv_file = 'temp_dataset.csv'
df.to_csv(csv_file, index=False)
dataset = load_dataset('csv', data_files=csv_file)
dataset['train'] = dataset['train'].select(range(1, len(dataset['train']))) # Remove the Open-Closed row

print(dataset['train'].column_names)

Generating train split: 0 examples [00:00, ? examples/s]

['ID', 'Question', 'Response', 'R2-1', 'R2_2B', 'R2_2D', 'R2_2SD', 'R2_3', 'R2_3YN', 'R2_OP', 'R2_4QG', 'R2_4QL', 'R2_4QP', 'R2_4QR', 'R2_4QI', 'R2_4QV', 'R2_5', 'R2_6']


In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from itertools import chain

# Combine Categories
# df['invitation'] = df[['R2_2B', 'R2_3YN', 'R2_4QP']].sum(axis=1)
# df['directive'] = df[['R2_2B', 'R2_3YN', 'R2_4QP']].sum(axis=1)
# df['option-posing'] = df[['R2_2B', 'R2_3YN', 'R2_4QP']].sum(axis=1)
# df['suggestive'] = df[['R2_2B', 'R2_3YN', 'R2_4QP']].sum(axis=1)

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
label_columns = ['R2-1', 'R2_2B', 'R2_2D', 'R2_2SD', 'R2_3', 'R2_3YN', 'R2_OP', 'R2_4QG', 'R2_4QL', 'R2_4QP', 'R2_4QR', 'R2_4QI', 'R2_4QV', 'R2_5','R2_6']

def tokenize_questions(examples):
    valid_questions = [q if q is not None else '' for q in examples['Question']]
    return tokenizer(valid_questions, padding=True, truncation=True)

# single label
def get_label(examples):
    label = -1
    for col in label_columns:
      try:
        if int(examples[col]) == 1:
          label = label_columns.index(col)
      except (ValueError, TypeError):
        print(f"Error processing column: {col}, column value: {examples[col]}, Question: {examples['Question']}")
        label = -1
    # if label == -1:
    #     print(f"No label found for Question: {examples['Question']}")
    examples['labels'] = torch.tensor(label)
    return examples

# multi label
# def encode_labels(examples):
#     labels = []
#     for col in label_columns:
#         try:
#             labels.append(int(examples[col]))  # Try to parse to int
#         except (ValueError, TypeError):
#             labels.append(0)  # Set to 0 if parsing fails
#     examples['labels'] = labels
#     return examples


# 1. Tokenize the questions
tokenized_dataset = dataset.map(tokenize_questions, batched=True)
# 2. Process labels
processed_datasets = tokenized_dataset.map(get_label, batched=False)

# 3. Remove unnecessary columns
other_columns_to_remove = ['ID', 'Response','Question']
# other_columns_to_remove = ['ID', 'Response']
processed_datasets = processed_datasets.remove_columns(label_columns + other_columns_to_remove)

# 4. Remove rows with no label
print(str(len(processed_datasets['train'])) + " rows including unlabled rows")
processed_datasets = processed_datasets.filter(lambda example: example['labels'] != -1)

print(str(len(processed_datasets['train']))+ " rows with labled rows")
print("Column names: "+ str(processed_datasets['train'].column_names))
print(processed_datasets['train']['labels'][:100])

# Many questions shows label 13, multilabel problem
# HM-HM shows label 1,2,5,13
for i in range(50):
    print(f"Question: {dataset['train'][i]['Question']}, Label: {processed_datasets['train'][i]['labels']}")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/2745 [00:00<?, ? examples/s]

Map:   0%|          | 0/2745 [00:00<?, ? examples/s]

Error processing column: R2_2B, column value:  , Question: OKAY THEN WHAT HAPPENED NEXT
2745 rows including unlabled rows


Filter:   0%|          | 0/2745 [00:00<?, ? examples/s]

2715 rows with labled rows
Column names: ['input_ids', 'token_type_ids', 'attention_mask', 'labels']
[0, 13, 0, 13, 13, 13, 13, 1, 13, 13, 2, 13, 13, 1, 13, 1, 13, 13, 1, 13, 1, 13, 3, 5, 4, 13, 5, 4, 5, 5, 5, 0, 13, 5, 5, 13, 13, 0, 10, 0, 13, 13, 2, 13, 1, 13, 1, 13, 1, 1, 13, 13, 13, 2, 4, 4, 2, 5, 3, 1, 5, 4, 1, 5, 3, 5, 13, 4, 4, 13, 3, 5, 5, 13, 5, 5, 13, 0, 0, 13, 5, 2, 0, 13, 4, 13, 4, 13, 13, 4, 13, 2, 13, 2, 13, 1, 13, 1, 13, 1]
Question: JESSICA TELL ME WHAT YOU'VE COME TO TALK TO ME ABOUT TODAY, Label: 0
Question: YES ABOUT CHURCH CAN YOU TELL ME WHEN YOU LAST WENT TO CHURCH EVERYTHING THAT HAPPENED THAT DAY, Label: 13
Question: START FROM THE BEGINNING, Label: 0
Question: YEAH, Label: 13
Question: AH-HA, Label: 13
Question: YEAH, Label: 13
Question: OKAY THEN WHAT HAPPENED NEXT, Label: 13
Question: OKAY, Label: 1
Question: OKAY AND THEN WHAT DID YOU DO, Label: 13
Question: YEAH, Label: 13
Question: HM-HM, Label: 2
Question: OKAY SO TELL ME ABOUT THAT PART, Label: 13
Questi

#### Using Dataset (failed)

In [ ]:

# !pip install transformers datasets pandas torch
# import pandas as pd
# from datasets import load_dataset, Dataset
# from torch.utils.data import DataLoader
# from transformers import AutoTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, DataCollatorWithPadding

# # Step 1: Load the dataset
# excel_file = 'Categorized_mocks.xlsx'
# df = pd.read_excel(excel_file, header=1)
# df = df.iloc[:, :18]
# csv_file = 'temp_dataset.csv'
# df.to_csv(csv_file, index=False)
# dataset = load_dataset('csv', data_files=csv_file)

# # Step 2: Define the custom dataset
# class QuestionTypeDataset(Dataset):
#     def __init__(self, dataframe):
#         self.dataframe = dataframe
#         self.label_columns = ['R2-1', 'R2_2B', 'R2_2D', 'R2_2SD', 'R2_3',
#                               'R2_3YN', 'R2_OP', 'R2_4QG', 'R2_4QL',
#                               'R2_4QP', 'R2_4QR', 'R2_4QI', 'R2_4QV',
#                               'R2_5', 'R2_6']

#     def process_label(self,item):
#       for col in self.label_columns:
#           if item[col] == 1:
#             return self.label_columns.index(col)
#       return -1  # Return -1 for no label found

#     def __len__(self):
#         return len(self.dataframe)

#     def __getitem__(self, idx):
#         item = self.dataframe[idx] # get item from dataset
#         question = item['Question']
#         label = self.process_label(item)

#         return {'question': question, # Return the question and label
#                 'label': label}

# # Step 3: Tokenization
# tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

# def tokenize_function(examples):
#   return tokenizer(examples['question'], padding=True, truncation=True, max_length=128)

# # Step 4: Split the dataset into training and test sets
# dataset = dataset['train'].train_test_split(test_size=0.2, seed=42)

# # Step 5: Tokenize the dataset
# tokenized_dataset = dataset.map(tokenize_function, batched=True,remove_columns=['ID', 'Response'])

# # Step 6: Convert to pytorch Dataset
# train_dataset = QuestionTypeDataset(tokenized_dataset['train'])
# test_dataset = QuestionTypeDataset(tokenized_dataset['test']) # Create a test dataset

# # Step 7: Define the BERT model
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(train_dataset.label_columns))

# # Step 8: Setup Trainer
# training_args = TrainingArguments(
#     output_dir='./results',
#     evaluation_strategy='epoch',
#     learning_rate=2e-5,
#     per_device_train_batch_size=16,
#     num_train_epochs=3,
# )

# # Step 9: Define data collator
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # Use DataCollatorWithPadding

# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset, # Add test dataset to the trainer
#     data_collator=data_collator, # Use data collator for padding
# )

# # Step 10: Train the model
# trainer.train()

In [ ]:
# from torch.utils.data import Dataset

# class QuestionTypeDataset(Dataset):

#     def __init__(self, data, tokenizer):
#         self.data = data
#         self.tokenizer = tokenizer

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         # print(self.data )
#         #  Dataset({
#         #     features: ['ID', 'Question', 'Response', 'R2-1', 'R2_2B', 'R2_2D', 'R2_2SD', 'R2_3', 'R2_3YN', 'R2_OP', 'R2_4QG', 'R2_4QL', 'R2_4QP', 'R2_4QR', 'R2_4QI', 'R2_4QV', 'R2_5', 'R2_6'],
#         #     num_rows: 2196
#         # })
#         # print(self.data[idx] )
#         # {'ID': 'NA0040-PC', 'Question': 'CAN YOU REMEMBER IF IT WAS IF HIS HAIR WAS LONG OR SHORT', 'Response': 'Short', 'R2-1': 0, 'R2_2B': '0', 'R2_2D': 0, 'R2_2SD': 0, 'R2_3': 0, 'R2_3YN': 1, 'R2_OP': 0, 'R2_4QG': 0, 'R2_4QL': 0, 'R2_4QP': 0, 'R2_4QR': 0, 'R2_4QI': 0, 'R2_4QV': 0, 'R2_5': 0, 'R2_6': 0}

#         item = self.data[idx]
#         label = self.get_label(item)

#         # Tokenize the text
#         question = self.tokenize_questions(item['Question'])

#         # Create a dictionary with input_ids and labels
#         return {'question':question, 'label':label}
#         # return {'input_ids':item['input_ids'],
#         #         'attention_mask':item['attention_mask'],
#         #         'label': self.process_label(item)}


#     def tokenize_questions(self,question):
#         question = question if question is not None else ''
#         # valid_question = [q if q is not None else '' for q in questions]
#         return tokenizer(question, padding=True, truncation=True)


#     def get_label(self,item):
#         for col in label_columns:
#             if item[col] is not None and item[col]=='' and int(item[col]) == 1:
#               return col
#         return ''


# train_dataset = QuestionTypeDataset(dataset['train'], tokenizer)
# eval_dataset = QuestionTypeDataset(dataset['test'], tokenizer)


#### 3. Prepare DataLoader
Convert the tokenized dataset into a PyTorch DataLoader for training and evaluation.

In [ ]:
from torch.utils.data import DataLoader

# 1. Split the dataset into training and test sets
processed_datasets = processed_datasets['train'].train_test_split(test_size=0.2, seed=42)

# 2. Get data_collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# 3. Get train_dataloader and eval_dataloader
train_dataloader = DataLoader(processed_datasets['train'], shuffle=True, batch_size=4, collate_fn=data_collator)
eval_dataloader = DataLoader(processed_datasets['test'], batch_size=4, collate_fn=data_collator)


#### 4. Set Up the Optimizer and Learning Rate Scheduler

In [ ]:
from transformers import AdamW, get_scheduler
import torch.optim as optim
import torch

optimizer = optim.AdamW(model.parameters(), lr=5e-5)

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    "linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

#### 5. Training Loop

##### 5.1 Method 1

In [ ]:
import transformers
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=15)

trainer = transformers.Trainer(
    model=model,
    train_dataset=processed_datasets['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorWithPadding(tokenizer=tokenizer)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
model.config.problem_type = "single_label_classification"
trainer.train()

max_steps is given, it will override any value given in num_train_epochs


RuntimeError: shape '[-1, 15]' is invalid for input of size 48

##### 5.2 Method 2

In [1]:
from tqdm.auto import tqdm
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=15)
model.to(device)
model.config.problem_type = "single_label_classification"

progress_bar = tqdm(range(num_training_steps))

# Option 1:
# for epoch in range(num_epochs):
#     model.train()
#     for batch in train_dataloader:
#         batch = {k: v.to(device) for k, v in batch.items()}
#         outputs = model(**batch)
#         loss = outputs.loss
#         loss.backward()

#         optimizer.step()
#         lr_scheduler.step()
#         optimizer.zero_grad()
#         progress_bar.update(1)


# Option 2: Accumulate Gradients
gradient_accumulation_steps = 8 # Example

for epoch in range(num_epochs):
    model.train()
    for i, batch in enumerate(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss / gradient_accumulation_steps # Normalize loss
        loss.backward()

        if (i+1) % gradient_accumulation_steps == 0: # Only update gradients every gradient_accumulation_steps
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()

        progress_bar.update(1)



NameError: name 'AutoModelForSequenceClassification' is not defined

#### 6. Evaluation Loop

In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        predictions = torch.argmax(outputs.logits, dim=-1)
        labels = batch["labels"]
        correct += (predictions == labels).sum().item()
        total += labels.size(0)

print(f"Accuracy: {correct / total:.4f}")
